In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import os
from difflib import SequenceMatcher


In [19]:
exp = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_expenditure.csv')

In [20]:
district_list = exp['Ddo Details'].str.split('/').str[0].dropna().unique().tolist()
department_list = exp['Ddo Details'].str.split('/').str[1].dropna().unique().tolist()

department_list

['GAD', 'DA', 'PWB', 'LRS', 'LRA', 'IGR', 'DMA']

In [1]:
district_map = {
    'LKM': 'Lakhimpur',                 # e.g. “134PHE/001 LKM ... LAKHIMPUR”
    'KAM': 'Kamrup',                    # e.g. “62 IRR/001 KAM ... Guwahati Irri Divn”
    'DBR': 'Dhubri',                    # e.g. “226PHE/001 DBR EX. ENGR. PHE, DHUBRI DIVN.”
    'BIS': 'Biswanath',                 # e.g. “224PHE/001 BIS EX. ENGR. PHE BISWANATH CHARIALI”
    'CAC': 'Cachar',                    # e.g. “357PHE/001 CAC EX. ENGR.PHE, SILCHAR DIV-I”
    'HKD': 'Hailakandi',                # e.g. “169PHE/001 HKD EX. ENGR.PHE DIV. HAILAKANDI”
    'BNG': 'Bongaigaon',                # e.g. “63 PHE/001 BNG E.E. PHE, BONGAIGAON”
    'NLB': 'Nalbari',                   # e.g. “443PHE/002 NLB EX. ENGR. PHE NALBARI DIVISION”
    'DIS': 'Dispur (Guwahati)',         # e.g. “5 PHE/001 DIS CHIEF ENGG. PHE, HENGRABARI, DISPUR”
    'NGT': 'Kamrup Metropolitan',       # “NGT” codes refer to Guwahati (e.g. “31 PHE/001 NGT ... GUWAHATI DIVN. NO.1”)
    'KAR': 'Karimganj',                 # e.g. “61 PHE/001 KAR EX. ENGR.PHE, KARIMGANJ”
    'BRT': 'Barpeta',                   # e.g. “231PHE/001 BRT EX. ENGINEER P.H.E., BARPETA DIVISION”
    'KKJ': 'Kokrajhar',                 # Common usage across multiple Bodoland references
    'DIB': 'Dibrugarh',                 # e.g. “98 PHE/002 DIB E.E. PHE. DIBRUGARH”
    'DMJ': 'Dhemaji',                   # e.g. “230PHE/001 DMJ ... DHEMAJI”
    'GLP': 'Goalpara',                  # e.g. “346PHE/001 GLP EX. ENGINEER P.H.E., GOALPARA”
    'NAG': 'Nagaon',                    # e.g. “105PHE/001 NAG SUPDT. ENG. .P.H.E, NAGAON DIVISION”
    'DAR': 'Darrang',                   # e.g. “373PHE/002 DAR EX. ENGINEER PHE , TANGLA DIVN (historically in Darrang)”
    'GT':  'Golaghat',                  # e.g. “24 PHE/001 GT EX.ENGR. P.H.E.,GOLAGHAT”
    'HAT': 'South Salmara Mankachar',   # “Hatsingimari” is HQ of the (sub)division
    'MRG': 'Morigaon',                  # e.g. “390PHE/001 MRG E.E.PHE, MORIGAON”
    'AKM': 'Karimganj',                 # No direct entry, but commonly used where “KAR” also stands for Karimganj
    'HFL': 'Dima Hasao',                # “Haflong” is the HQ
    'GAR': 'Majuli',                    # “Garamur” in Majuli (e.g. “308FC/001 GAR EX. ENGR. FLOOD CONTROL, GARAMUR”)
    'TSK': 'Tinsukia',                  # e.g. “296PHE/001 TSK EX. ENGR. PHE, TINSUKIA”
    'UDG': 'Udalguri',                  # e.g. “512PHE/001 GOS … also references Tangla, etc. in Udalguri
    'MSL': 'Majuli',                    # No direct mention, but often used as “Mājuli” code
    'DPU': 'Karbi Anglong',             # “Diphu” is HQ of Karbi Anglong
    # The second 'KAM' is presumably the same Kamrup
    'SVS': 'Sivasagar',                 # e.g. “111PHE/001 SVS EX. ENGINEER PHE, SIVASAGAR”
    'JOR': 'Jorhat',                    # e.g. “83 PHE/001 JOR S.E.PHE, JORHAT”
    'HOJ': 'Hojai',                     # e.g. “393PHE/001 HOJ EX. ENGR. PHE, HOJAI DIV.”
    'HAM': 'West Karbi Anglong',        # “Hamren” is the HQ of West Karbi Anglong
    'SON': 'Sonitpur',                  # e.g. “221PHE/002 SON E.E. P.H.E. TEZPUR DIVISION …”
    'SNR': 'Charaideo',                 # e.g. “250PWR/001 SNR EX. ENGR. RURAL ROAD DIV, CHARAIDEO”
    'SLK': 'South Salmara Mankachar',   # Another short code used for Salmara–Mankachar
    'PSL': 'Bajali',                    # “Pathsala” is HQ for newly formed Bajali
    'TML': 'Tamulpur',                  # Another new district
    'SRT': 'Bajali'                     # Possibly “Sarthebari”/“Sarupeta,” but effectively also in Bajali
}


## Merge on DDO file

In [21]:
#Write a script that identifies the closes match between entries of a datarame
ddo_code = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\ddo_treasurycode_deptname_cleaned.csv')


In [22]:
merged = pd.merge(exp, ddo_code[['TREASURY CODE','Head Office','TREASURY NAME','DDO NAME']], how='left', left_on='Ddo Details', right_on='TREASURY CODE')
merged

,Name of the Scheme,Whether scheme,Whether the scheme is funded by NABARD,Proposal Number,Proposal Date,Issued Number,Valid Upto,Admin Department File Number,Brief nature of the scheme,"Aim, objectives and benefit expected from the scheme/project",...,Season,Scheme,Erosion,Roads_Bridges_Embkt,Response Type,Flood Response - Subhead,TREASURY CODE,Head Office,TREASURY NAME,DDO NAME
0,Immediate measures for closing of breach and r...,New project,No,AA-05-2019-20-0197,06-06-2019,AA/05_2019-20(I)_11,20-07-2022,RGR(RRR)386/2019,Immediateestoration of flood damages of river ...,To protect the people of the greater Dhakuakha...,...,Monsoon,SDRF,False,True,Immediate Measures,"{'sdrf': 1, 'immediate': 1}",LKM/GAD/001,North Lakhimpur,Lakhimpur,"DEPUTY COMMISSIONER,LAKHIMPUR"
1,Immediate measures for restoration of damaged ...,New project,No,AA-05-2019-20-0198,06-06-2019,AA/05_2019-20(I)_10,19-07-2022,RGr(RRR)388/2019,Immediate measures for repairing and restoration.,To protect the people of Greater Dhakuakhana area,...,Monsoon,SDRF,False,False,Immediate Measures,"{'sdrf': 1, 'immediate': 2}",LKM/GAD/001,North Lakhimpur,Lakhimpur,"DEPUTY COMMISSIONER,LAKHIMPUR"
2,Restoration of flood damaged road from 6th KM ...,New project,No,AA-05-2019-20-0200,14-06-2019,AA/05_2019-20(I)_01,14-06-2022,RGR(RRR)951/2018,Restoration of flood damaged work,"It is the only road for movement to schools, c...",...,Monsoon,SDRF,False,True,Immediate Measures,{'sdrf': 1},DBR/GAD/001,Dhubri,Dhubri,"DEPUTY COMMISSIONER, DHUBRI"
3,Immediate measures for restoration of damages ...,New project,No,AA-05-2019-20-0205,26-06-2019,AA/05_2019-20(I)_28,06-12-2022,RGR(RRR)548/2019,The proposed scheme has been framed to prevent...,The proposed scheme has been framed to prevent...,...,Monsoon,NaN,True,True,Immediate Measures,{'immediate': 1},BIS/GAD/001,Biswanath Chariali,Biswanath Chariali,DEPUTY COMMISSIONER B. CHARIALI
4,"""Immediate measures to Ring Bund at villagae T...",New project,No,AA-05-2019-20-0206,26-06-2019,AA/05_2019-20(I)_09,19-07-2024,RGR(RRR)819/2018,NaN,To protect the life and property of the people...,...,Monsoon,SDRF,False,True,Immediate Measures,"{'sdrf': 1, 'immediate': 1}",CAC/GAD/001,Silchar,Cachar,"DEPUTY COMMISSIONER, CACHAR"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6379,CONSTRUCTION OF FRONT BUILDING (GROUND + THREE...,Ongoing Project(Parent Proposal No. AA-SDM-202...,NaN,NaN,06-12-2024,NaN,NaN,-N/A-,-N/A-,-N/A-,...,Monsoon,NaN,False,False,Others,NaN,NaN,NaN,NaN,NaN
6380,CONSTRUCTION OF FRONT BUILDING (GROUND + THREE...,Ongoing Project(Parent Proposal No. AA-SDM-202...,NaN,NaN,17-12-2024,NaN,NaN,-N/A-,-N/A-,-N/A-,...,Post-Monsoon,NaN,False,False,Others,NaN,NaN,NaN,NaN,NaN
6381,Procurement of Machinery and Trolley Mounted P...,New project,No,AA-SDM-2024-25-0007,20-01-2025,AA/SDM_2024-25(I)_03,29-01-2030,499824,Procurement of Machinery and Trolley Mounted P...,1. The pumps will aid in mitigation of urban f...,...,Pre-Monsoon,SDRF,False,True,Immediate Measures,{'sdrf': 2},NaN,NaN,NaN,NaN
6382,"Construction of DEOC building, Kamrup, Amingao...",New project,No,AA-SDM-2024-25-0008,24-02-2025,AA/SDM_2024-25(I)_04,11-03-2028,NaN,"Construction of DEOC building, Kamrup, Amingao...","Construction of DEOC building, Kamrup, Amingao...",...,Pre-Monsoon,NaN,False,False,Others,NaN,NaN,NaN,NaN,NaN


In [23]:
merged.to_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_DDO_expenses.csv')

In [17]:
unique_treasury = merged['TREASURY NAME'].unique()
print(unique_treasury)
print("Count:", len(unique_treasury))

['Lakhimpur' 'Dhubri' 'Biswanath Chariali' 'Cachar' 'Hailakandi'
 'Bongaigaon' 'Nalbari' 'Dispur' 'New-Guwahati' 'Karimganj' 'Barpeta'
 'Kokrajhar' 'Dibrugarh' 'Dhemaji' 'Goalpara' 'Nagaon' 'Darrang'
 'Golaghat' 'Hat Singimari' 'Marigaon' 'Kamrup,Amingaon' nan 'Haflong'
 'Garamur' 'Tinsukia' 'Udalguri' 'Mushalpur' 'Diphu' 'Kamrup Metro'
 'Sivasagar' 'Jorhat' 'Hojai' 'Hamren' 'Sonitpur' 'Sonari' 'Pathsala']
Count: 36


In [16]:
rc_gdf = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\as_ids-drr_shapefiles\assam_rc_2024-11.geojson')

In [ ]:

fuzzymatch = fuzzy_merge(rc_gdf, melted_df, 'dtname', 'District', threshold=80,limit=1)
fuzzymatch

In [34]:
sdrf_df = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_DDO_expenses.csv')


### TRADITIONAL METHOD

In [ ]:
import os
import re
import pandas as pd
import geopandas as gpd
from difflib import SequenceMatcher
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# The erroneous assignment has been removed so that load_data() returns the full DataFrame.


# ---------------- Global Dictionaries ---------------- #

# Dictionary for correcting specific village name typos
VILLAGE_CORRECTION_DICT = {
    "SOKARBILA(BOLGARBARI)(DARIAPAR": "SOKARBILA(BOLGARBARI)(DARIAPAR)",
    "MANGALDAI EXTENDED TOWN (BHEBA": "MANGALDAI EXTENDED TOWN (BHEBA)",
    "UPPER DIHING R.F. (SOUTH BLOCK": "UPPER DIHING R.F. (SOUTH BLOCK)",
    "KACHARI MAITHCHAGAON NO.1(BAR": "KACHARI MAITHCHAGAON NO.1(BAR)",
}

# Dictionary for mapping three-letter district identifiers to proper district names
three_letter_distirct_identifiers_dict = {
    "bak": "BAKSA", "baksa": "BAKSA",
    "bar": "BARPETA", "re-bar": "BARPETA", "barpeta": "BARPETA",
    "bongaigoan": "BONGAIGAON",
    "tez": "SONITPUR", "re-tez": "SONITPUR", "tezpur": "SONITPUR", "tej": "SONITPUR", "re-tej": "SONITPUR",
    "silchar": "CACHAR", "re-silchar": "CACHAR", "resilchar": "CACHAR",
    "re-sil(mech)": "CACHAR", "silchar (mech)": "CACHAR", "sil": "CACHAR",
    "sil (mech)": "CACHAR", "sil(mech)": "CACHAR",
    "dhubri": "DHUBRI", "dhu": "DHUBRI",
    "siv": "SIVASAGAR", "sivsagar": "SIVASAGAR", "re-siv": "SIVASAGAR", "sivasagar": "SIVASAGAR",
    "chirang": "CHIRANG",
    "mang": "DARRANG", "re-mang": "DARRANG", "mangaldai": "DARRANG", "mangaldoi": "DARRANG",
    "dhe": "DHEMAJI", "dhemaji": "DHEMAJI", "dmj": "DHEMAJI", "redhemaji": "DHEMAJI",
    "hailakandi": "HAILAKANDI", "hkd": "HAILAKANDI", "re-hailakandi": "HAILAKANDI",
    "dib-west": "DIBRUGARH", "dib": "DIBRUGARH", "dibrugarh": "DIBRUGARH", "redib": "DIBRUGARH",
    "dima-hasao": "DIMA HASAO", "haf": "DIMA HASAO", "haflong": "DIMA HASAO",
    "goalpara": "GOALPARA", "GLP": "GOALPARA",
    "diphu": "KARBI ANGLONG", "rediphu": "KARBI ANGLONG",
    "jor": "JORHAT", "jorhat": "JORHAT",
    "nag": "NAGAON", "re-nag": "NAGAON", "nagaon": "NAGAON", "hatimura": "NAGAON",
    "nal": "NALBARI", "nalbari": "NALBARI",
    "morigaon": "MORIGAON", "mor": "MORIGAON", "re-mor": "MORIGAON",
    "maj": "MAJULI", "re-maj": "MAJULI", "maju": "MAJULI", "majuli": "MAJULI",
    "n.lakhimpur": "LAKHIMPUR", "dhakuakhana": "LAKHIMPUR", "nlp": "LAKHIMPUR", "nl": "LAKHIMPUR", "dhk": "LAKHIMPUR",
    "kar": "KARIMGANJ", "rekar": "KARIMGANJ", "re-kar": "KARIMGANJ", "karimganj": "KARIMGANJ", "badarpur": "KARIMGANJ",
    "gmda": "KAMRUP METRO", "ghy east": "KAMRUP METRO", "ghy.east": "KAMRUP METRO",
    "ghy. east": "KAMRUP METRO", "ghyeast": "KAMRUP METRO", "g.east": "KAMRUP METRO",
    "ghy west": "KAMRUP METRO", "ghy.west": "KAMRUP METRO", "ghy. west": "KAMRUP METRO", "ghywest": "KAMRUP METRO",
    "kok": "KOKRAJHAR",
    "rangia": "KAMRUP",
    "gdd": "KAMRUP METRO"
}

# ---------------- Function Definitions ---------------- #

def load_data():
    """
    Loads tender data, village master data, and revenue circle geospatial data.
    Adjust file paths as necessary.
    """
    print(os.getcwd())
    expense_path = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_DDO_expenses.csv'
    villages_path = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\ASSAM_VILLAGES_MASTER.csv'
    rcs_path = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\as_ids-drr_shapefiles\assam_rc_2024-11.geojson'
    
    sdrf_df = pd.read_csv(expense_path)
    ASSAM_VILLAGES = pd.read_csv(villages_path, encoding='utf-8').dropna()
    ASSAM_RCS = gpd.read_file(rcs_path, driver='GeoJSON')
    return sdrf_df, ASSAM_VILLAGES, ASSAM_RCS


def geocode_district(sdrf_df, ASSAM_VILLAGES):
    """
    Performs initial geocoding at the district level.
    First, a fuzzy-match corrects the tender 'location' field against revenue circle names.
    Then, using multiple text fields (external reference, title/description, location), 
    candidate district assignments are made and a weightage logic finalizes the district.
    """
    # --- Fuzzy-match location to revenue circle names ---
    unique_rcs = ASSAM_VILLAGES['revenue_ci'].dropna().unique()
    locations = []
    for idx, row in sdrf_df.iterrows():
        loc_clean = str(row['TREASURY NAME']).lower().replace('Chariali','').replace('Kamrup,','').replace('dist','')
        loc_clean = re.sub(r'[^a-zA-Z\n\.]', ' ', loc_clean)
        scores = [SequenceMatcher(None, loc_clean, rc.lower().strip()).ratio() for rc in unique_rcs]
        if scores and max(scores) > 0.8:
            best_match = unique_rcs[scores.index(max(scores))]
            locations.append(best_match)
        else:
            locations.append(row['TREASURY NAME'])
    sdrf_df['TREASURY NAME'] = locations

    # --- Prepare lists for matching ---
    assam_districts = list(set(ASSAM_VILLAGES['district_2'].dropna()) - set(['KAMRUP','KAMRUP METRO']))
    assam_revenue_circles = list(set(ASSAM_VILLAGES['revenue_ci'].dropna()))
    assam_sub_districts = list(set(ASSAM_VILLAGES['sdtname_2'].dropna()))
    
    # Build dictionaries for revenue circles and sub-districts (used later)
    assam_revenue_circles_dict = (ASSAM_VILLAGES[['revenue_ci','district_2']]
                                  .dropna().drop_duplicates().drop_duplicates(['revenue_ci'], keep='first')
                                  .set_index('revenue_ci').to_dict(orient='index'))
    assam_subdist_dict = (ASSAM_VILLAGES[['sdtname_2','district_2']]
                          .dropna().drop_duplicates().drop_duplicates(['sdtname_2'], keep='first')
                          .set_index('sdtname_2').to_dict(orient='index'))
    
    # --- Assign district candidates from multiple sources ---
    sdrf_df = assign_district_candidates(sdrf_df, assam_districts, assam_revenue_circles,
                                         assam_sub_districts, assam_revenue_circles_dict,
                                         assam_subdist_dict, three_letter_distirct_identifiers_dict)
    return sdrf_df


def assign_district_candidates(sdrf_df, assam_districts, assam_revenue_circles, assam_sub_districts,
                               assam_revenue_circles_dict, assam_subdist_dict, three_letter_dict):
    """
    For each tender, this function generates candidate district assignments from:
      - The tender external reference
      - The tender title and Aim, objectives and benefit expected from the scheme/project
      - The tender location
    Then it uses a simple weightage logic to finalize the district.
    """
    # Candidate 1: Based on external reference
    sdrf_df['expense_district_externalReference'] = None
    for idx, row in sdrf_df.iterrows():
        ref = str(row['DDO Code'])
        district_identifier = ref.split('/')[0].lower()
        if 'rgr' in district_identifier:
            district_identifier = district_identifier.split('rgr')[0].strip()[:-1]
        if district_identifier in three_letter_dict:
            sdrf_df.loc[idx, 'expense_district_externalReference'] = three_letter_dict[district_identifier]

    # Candidate 2: Based on title and Aim, objectives and benefit expected from the scheme/project
    sdrf_df['tender_district_title_description'] = None
    for idx, row in sdrf_df.iterrows():
        expense_slug = f"{row['Name of the Scheme']} {row['Aim, objectives and benefit expected from the scheme/project']}"
        expense_slug = re.sub(r'[^a-zA-Z0-9 \n\.]', ' ', expense_slug)
        # Try direct district match
        for district in assam_districts:
            if re.search(r'\b' + re.escape(district.lower().strip()) + r'\b', expense_slug.lower()):
                sdrf_df.loc[idx, 'tender_district_title_description'] = district
                break
        # Fallback to matching revenue circles
        if sdrf_df.loc[idx, 'tender_district_title_description'] is None:
            for rc in assam_revenue_circles:
                if re.search(r'\b' + re.escape(rc.lower().strip()) + r'\b', expense_slug.lower()):
                    if rc in assam_revenue_circles_dict:
                        sdrf_df.loc[idx, 'tender_district_title_description'] = assam_revenue_circles_dict[rc]['district_2']
                    else:
                        # Optionally, assign the revenue circle itself or leave as NA
                        sdrf_df.loc[idx, 'tender_district_title_description'] = rc
                    break

        # Fallback to matching sub-districts
        if sdrf_df.loc[idx, 'tender_district_title_description'] is None:
            for sd in assam_sub_districts:
                if re.search(r'\b' + re.escape(sd.lower()) + r'\b', expense_slug.lower()):
                    sdrf_df.loc[idx, 'tender_district_title_description'] = assam_subdist_dict[sd]['district_2']
                    break

    # Candidate 3: Based on location
    sdrf_df['tender_district_location'] = None
    for idx, row in sdrf_df.iterrows():
        expense_slug = str(row['TREASURY NAME'])
        expense_slug = re.sub(r'[^a-zA-Z0-9 \n\.]', ' ', expense_slug)
        for district in assam_districts:
            if re.search(r'\b' + re.escape(district.lower().strip()) + r'\b', expense_slug.lower()):
                sdrf_df.loc[idx, 'tender_district_location'] = district
                break
        if sdrf_df.loc[idx, 'tender_district_location'] is None:
            for rc in assam_revenue_circles:
                if re.search(r'\b' + re.escape(rc.lower().strip()) + r'\b', expense_slug.lower()):
                    if rc in assam_revenue_circles_dict:
                        sdrf_df.loc[idx, 'tender_district_title_description'] = assam_revenue_circles_dict[rc]['district_2']
                    else:
                        # Optionally, assign the revenue circle itself or leave as NA
                        sdrf_df.loc[idx, 'tender_district_title_description'] = rc
                    break

        if sdrf_df.loc[idx, 'tender_district_location'] is None:
            for sd in assam_sub_districts:
                if re.search(r'\b' + re.escape(sd.lower()) + r'\b', expense_slug.lower()):
                    sdrf_df.loc[idx, 'tender_district_location'] = assam_subdist_dict[sd]['district_2']
                    break

    # Finalize district using weightage logic
    sdrf_df['expense_district_externalReference'].fillna('NA', inplace=True)
    sdrf_df['tender_district_title_description'].fillna('NA', inplace=True)
    sdrf_df['tender_district_location'].fillna('NA', inplace=True)
    sdrf_df['DISTRICT_FINALISED'] = ''
    for idx, row in sdrf_df.iterrows():
        districts = {row['expense_district_externalReference'],
                     row['tender_district_title_description'],
                     row['tender_district_location']}
        districts = {d for d in districts if d != 'NA'}
        if len(districts) == 1:
            sdrf_df.loc[idx, 'DISTRICT_FINALISED'] = list(districts)[0]
        elif len(districts) == 0:
            sdrf_df.loc[idx, 'DISTRICT_FINALISED'] = 'NA'
        else:
            sdrf_df.loc[idx, 'DISTRICT_FINALISED'] = 'CONFLICT'
    return sdrf_df


def geocode_details(tender_df, ASSAM_VILLAGES, RC_HQs):
    """
    For each district (FOCUS_DISTRICT), this function builds district-specific dictionaries 
    (villages, blocks, revenue circles, subdistricts) from the master villages data.
    Then it matches tender text (a combination of tender fields) against these units to assign
    detailed geocoding (village, block, subdistrict, revenue circle). Finally, it uses the revenue
    circle HQ information to finalize the revenue circle assignment.
    """
    MASTER_DF_list = []
    
    # Process each district from the master village data
    for FOCUS_DISTRICT in tqdm(ASSAM_VILLAGES['district_2'].unique(), desc="Detail Geocoding"):
        district_df = ASSAM_VILLAGES[ASSAM_VILLAGES['district_2'] == FOCUS_DISTRICT]
        FOCUSDIST_village_dict = {}
        FOCUSDIST_block_dict = {}
        FOCUSDIST_subdistrict_dict = {}
        FOCUSDIST_revcircle_dict = {}
        
        for idx, row in district_df.iterrows():
            # Clean village name and correct if needed
            if row["VILNAM_SOI"]:
                village_clean = re.sub(r'[^a-zA-Z]', "", row["VILNAM_SOI"])
                if village_clean in VILLAGE_CORRECTION_DICT:
                    village_clean = VILLAGE_CORRECTION_DICT[village_clean]
                FOCUSDIST_village_dict[village_clean] = {
                    "village_id": row["OBJECTID"],
                    "block_name": row["block_name"],
                    "subdistrict": row["sdtname_2"],
                    "revenuecircle": row["revenue_ci"],
                    "district_2": row["district_2"]
                }
            FOCUSDIST_block_dict[row["block_name"]] = {
                "subdistrict": row["sdtname_2"],
                "revenuecircle": row["revenue_ci"],
                "district_2": row["district_2"]
            }
            FOCUSDIST_subdistrict_dict[row["sdtname_2"]] = {"district_2": row["district_2"]}
            FOCUSDIST_revcircle_dict[row["revenue_ci"]] = {"district_2": row["district_2"]}
        
        # Remove any unwanted keys if present
        for key in ['RIVER', 'NO', 'TOWN']:
            FOCUSDIST_village_dict.pop(key, None)
        
        FOCUSDIST_villages = list(FOCUSDIST_village_dict.keys())
        FOCUSDIST_blocks = list(FOCUSDIST_block_dict.keys())
        FOCUSDIST_subdistricts = list(FOCUSDIST_subdistrict_dict.keys())
        FOCUSDIST_revcircles = list(FOCUSDIST_revcircle_dict.keys())
        
        # Subset tenders geocoded to this district
        tender_subset = tender_df[tender_df["DISTRICT_FINALISED"] == FOCUS_DISTRICT].copy()
        
        # Define a regex pattern to remove specific substrings
        substrings_to_remove = ["(pt)", "\n"]
        pattern = "|".join(map(re.escape, substrings_to_remove))
        
        # Process each tender in the focused district
        for idx, row in tender_subset.iterrows():
            expense_slug = f"{row['Name of the Scheme']} {row['Aim, objectives and benefit expected from the scheme/project']}"
            expense_slug = re.sub(r'[^a-zA-Z0-9 \n\.]', ' ', expense_slug)
            
            tender_villages = []
            tender_village_id = ""
            tender_block = ""
            tender_revenueci = ""
            tender_subdistrict = ""
            tender_revenueci_location = ""
            
            # Match village names
            for village in FOCUSDIST_villages:
                if not re.search(r'[a-zA-Z]', village):
                    continue 
                village_search = re.sub(pattern, " ", village.lower())
                if re.search(r'\b' + re.escape(village_search.strip()) + r'\b', expense_slug.lower()):
                    tender_villages.append(village)
                    tender_village_id = FOCUSDIST_village_dict[village]['village_id']
                    tender_block = FOCUSDIST_village_dict[village]['block_name']
                    tender_revenueci = FOCUSDIST_village_dict[village]['revenuecircle']
                    tender_subdistrict = FOCUSDIST_village_dict[village]['subdistrict']
            
            # If no village found, try matching block names
            if not tender_block:
                for block in FOCUSDIST_blocks:
                    block_search = re.sub(pattern, " ", block.lower())
                    if re.search(r'\b' + re.escape(block_search.strip()) + r'\b', expense_slug.lower()):
                        tender_block = block
                        tender_revenueci = FOCUSDIST_block_dict[block]['revenuecircle']
                        tender_subdistrict = FOCUSDIST_block_dict[block]['subdistrict']
                        break
            
            # Try matching revenue circle from the tender slug
            for revenue_circle in FOCUSDIST_revcircles:
                revenue_circle_search = re.sub(pattern, " ", revenue_circle.lower())
                if re.search(r'\b' + re.escape(revenue_circle_search.strip()) + r'\b', expense_slug.lower()):
                    tender_revenueci = revenue_circle
                    break
            
            # Try matching revenue circle from the tender 'location' field
            for revenue_circle in FOCUSDIST_revcircles:
                revenue_circle_search = re.sub(pattern, " ", revenue_circle.lower())
                if re.search(r'\b' + re.escape(revenue_circle_search.strip()) + r'\b', str(row['TREASURY NAME']).lower()):
                    tender_revenueci_location = revenue_circle
                    break
            
            # Try matching subdistrict from the tender slug
            for subdistrict in FOCUSDIST_subdistricts:
                subdistrict_search = re.sub(pattern, " ", subdistrict.lower())
                if re.search(r'\b' + re.escape(subdistrict_search.strip()) + r'\b', expense_slug.lower()):
                    tender_subdistrict = subdistrict
                    break
            
            # Save matched details into the dataframe
            tender_subset.loc[idx, 'tender_villages'] = ','.join(tender_villages)
            tender_subset.loc[idx, 'tender_block'] = tender_block
            tender_subset.loc[idx, 'tender_subdistrict'] = tender_subdistrict
            tender_subset.loc[idx, 'tender_revenueci'] = tender_revenueci
            tender_subset.loc[idx, 'tender_revenueci_location'] = tender_revenueci_location
        
        MASTER_DF_list.append(tender_subset)
    
    # Append tenders that did not get a clear district assignment
    MASTER_DF_list.append(tender_df[tender_df["DISTRICT_FINALISED"] == 'NA'])
    MASTER_DF_list.append(tender_df[tender_df["DISTRICT_FINALISED"] == 'CONFLICT'])
    
    MASTER_DF = pd.concat(MASTER_DF_list, ignore_index=True)
    
    # Finalize revenue circle using HQ flag from revenue circle geo-data
    MASTER_DF['HQ_flag'] = False
    MASTER_DF['REVENUE_CIRCLE_FINALISED'] = ''
    for idx, row in MASTER_DF.iterrows():
        if row.get('tender_revenueci_location', '') in RC_HQs:
            MASTER_DF.loc[idx, 'HQ_flag'] = True
        if row.get('tender_revenueci_location', ''):
            MASTER_DF.loc[idx, 'REVENUE_CIRCLE_FINALISED'] = row['tender_revenueci_location']
        else:
            MASTER_DF.loc[idx, 'REVENUE_CIRCLE_FINALISED'] = row.get('tender_revenueci', '')
        if row.get('tender_revenueci_location', '') == row.get('tender_revenueci', ''):
            MASTER_DF.loc[idx, 'REVENUE_CIRCLE_FINALISED'] = row['tender_revenueci']
    
    return MASTER_DF


def main():
    """
    Main function to run the consolidated geocoding workflow.
    It loads data, applies district geocoding, then detailed geocoding,
    and finally writes the geotagged output to CSV.
    """
    # Load raw data
    sdrf_df, ASSAM_VILLAGES, ASSAM_RCS = load_data()
    
    # Perform district-level geocoding
    sdrf_df = geocode_district(sdrf_df, ASSAM_VILLAGES)
    
    # (Optional) Write intermediate district-geotagged data
    district_out_path = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_expenses_RCgeotagged.csv'
    sdrf_df.to_csv(district_out_path, index=False)
    
    # Get list of Revenue Circle HQs from the revenue circle geodata
    RC_HQs = list(ASSAM_RCS[ASSAM_RCS['HQ'] == 'y']['revenue_ci'])
    
    # Perform detailed geocoding (villages, blocks, etc.) based on finalized district tags
    MASTER_DF = geocode_details(sdrf_df, ASSAM_VILLAGES, RC_HQs)
    
    # Write final geocoded output
    final_out_path = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_expenses_RCgeotagged.csv'
    MASTER_DF.to_csv(final_out_path, index=False)
    
    # Print summary statistics
    total = sdrf_df.shape[0]
    na_count = sdrf_df[sdrf_df['DISTRICT_FINALISED'] == 'NA'].shape[0]
    conflict_count = sdrf_df[sdrf_df['DISTRICT_FINALISED'] == 'CONFLICT'].shape[0]
    print('Total number of flood related tenders:', total)
    print('Number of tenders whose district could not be geo-tagged:', na_count)
    print('Number of tenders with district identification conflicts:', conflict_count)


if __name__ == '__main__':
    main()


d:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\scripts


Detail Geocoding: 100%|██████████| 35/35 [02:40<00:00,  4.60s/it]


Total number of flood related tenders: 6384
Number of tenders whose district could not be geo-tagged: 1137
Number of tenders with district identification conflicts: 404


In [4]:
exp_tagged = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_expenses_RCgeotagged.csv')
dist_gdf = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\as_ids-drr_shapefiles\assam_district_2024-11.geojson', driver='GeoJSON')

In [17]:
exp_dist = set(exp_tagged['DISTRICT_FINALISED'].unique())
gdf_dist = set(dist_gdf['dtname'].unique())

In [ ]:
diff = gdf_dist - exp_dist
diff

{'CHIRANG', 'KARBI ANGLONG', 'LAKHIMPUR', 'SIVASAGAR', 'TAMULPUR'}

In [ ]:
diff = exp_dist - gdf_dist
diff

{'CONFLICT', 'K.ANGLONG', 'SIVSAGAR', nan}

Unique entries: ['HAILAKANDI' 'KARIMGANJ' 'CACHAR' 'SOUTH SALMARA MANCACHAR' 'HOJAI'
 'DIMA HASAO' 'WEST KARBI ANGLONG' 'KAMRUP' 'K.ANGLONG' 'DHUBRI'
 'GOALPARA' 'NAGAON' 'GOLAGHAT' 'KAMRUP METRO' 'MORIGAON' 'BARPETA'
 'KOKRAJHAR' 'BONGAIGAON' 'NALBARI' 'DARRANG' 'JORHAT' 'BAJALI' 'BAKSA'
 'UDALGURI' 'SONITPUR' 'BISWANATH' 'SIVASAGAR' 'MAJULI' 'CHARAIDEO'
 'DIBRUGARH' 'TINSUKIA' 'DHEMAJI'], Count: 34 #nan 'CONFLICT'

In [ ]:
districts = exp['DISTRICT_FINALISED'].unique()




# List unique entries in "DISTRICT_FINALISED" and count them
unique_entries = df['DISTRICT_FINALISED'].unique()
unique_count = len(unique_entries)
# Create a DataFrame with the result
df = pd.DataFrame({'Answer': [f"Unique entries: {unique_entries}, Count: {unique_count}"]})

### Test with Sentence-transformers

In [25]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# --------------------------------------------------
# Helper Functions
# --------------------------------------------------

def clean_text(text):
    """
    Lowercases and removes extra whitespace from text.
    Returns an empty string if the input is not a string.
    """
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # collapse multiple spaces
    return text.strip()

def combine_fields(row, fields):
    """
    Combines specified fields from a DataFrame row into a single string.
    """
    parts = []
    for field in fields:
        parts.append(str(row.get(field, "")))
    return " ".join(parts)

def compute_embeddings(model, texts):
    """
    Computes embeddings for a list of texts using the provided SentenceTransformer model.
    """
    return model.encode(texts, convert_to_tensor=False)

def geocode_district(query_embedding, district_embeddings, district_names, conflict_threshold=0.1):
    """
    Given a scheme embedding, computes cosine similarity with candidate district embeddings.
    Returns the best matching district, its similarity score, and a warning string if the top two
    similarities are within 'conflict_threshold' (indicating ambiguity).
    """
    # Compute cosine similarities: query_embedding should be shaped as (1, D)
    sims = cosine_similarity(query_embedding, district_embeddings)[0]
    best_idx = np.argmax(sims)
    best_sim = sims[best_idx]
    best_district = district_names[best_idx]
    
    # Get second-best similarity (if available)
    sims_sorted = np.sort(sims)[::-1]
    second_best_sim = sims_sorted[1] if len(sims_sorted) > 1 else 0
    
    warning = ""
    if best_sim - second_best_sim < conflict_threshold:
        warning = f"Conflict: top similarities are {best_sim:.2f} and {second_best_sim:.2f}"
        best_district = "CONFLICT"
    
    return best_district, best_sim, warning

def geocode_revenue_circle(query_embedding, revenue_embeddings, revenue_names):
    """
    Computes cosine similarity between the query embedding and candidate revenue circle embeddings.
    Returns the best matching revenue circle and its similarity score.
    """
    sims = cosine_similarity(query_embedding, revenue_embeddings)[0]
    best_idx = np.argmax(sims)
    best_sim = sims[best_idx]
    best_revenue = revenue_names[best_idx]
    return best_revenue, best_sim

In [ ]:
# --------------------------------------------------
# Main Geocoding Function
# --------------------------------------------------

def main():
    """
    Main function that:
      1. Loads scheme data and the village master CSV (which includes district and revenue circle info).
      2. Uses an embedding-based approach to match each scheme to a district.
      3. If a unique district is identified, it then matches to a revenue circle within that district.
      4. Flags a conflict if the district matching is ambiguous.
      5. Saves the final geocoded output.
    """
    # ----------------------------
    # Load Data
    # ----------------------------
    # Adjust these paths as needed
    schemes_csv = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_DDO_expenses.csv"         # Your schemes CSV file
    village_csv = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\ASSAM_VILLAGES_MASTER.csv"      # CSV with columns: OBJECTID, VILNAM_SOI, block_name, sdtname_2, revenue_ci, district_2, object_id, district_1, revenue_cr
    rc_gdf = gpd.read_file(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\as_ids-drr_shapefiles\assam_rc_2024-11.geojson")  # Shapefile for revenue circles

    schemes_df = pd.read_csv(schemes_csv)
    village_df = pd.read_csv(village_csv)
    
    # ----------------------------
    # Prepare Unique District List
    # ----------------------------
    # We use the district_2 column from the village CSV
    unique_districts = rc_gdf["DISTRICT_FINALISED"].dropna().unique().tolist()
    unique_districts = [clean_text(x) for x in unique_districts]
    
    # ----------------------------
    # Load SentenceTransformer Model
    # ----------------------------
    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # Compute embeddings for unique districts
    district_embeddings = compute_embeddings(model, unique_districts)
    
    # ----------------------------
    # Set up storage for geocoding results
    # ----------------------------
    predicted_districts = []
    district_similarity_scores = []
    district_warnings = []
    
    predicted_revenue_circles = []
    revenue_similarity_scores = []
    
    # Define which fields to combine for geocoding
    text_fields = [
        "Name of the Scheme", 
        "Head office", 
        "Aim, objectives and benefit expected from the scheme/project", 
        "TREASURY NAME", 
        "DDO NAME"
    ]
    
    # ----------------------------
    # Process each scheme
    # ----------------------------
    for idx, row in schemes_df.iterrows():
        # Combine the key text fields into one string
        combined_text = combine_fields(row, text_fields)
        combined_text_clean = clean_text(combined_text)
        
        # Compute the embedding for the combined text (shape: (1, D))
        query_embedding = model.encode([combined_text_clean], convert_to_tensor=False)
        
        # ----- District-Level Geocoding -----
        pred_district, district_sim, warning = geocode_district(query_embedding, district_embeddings, unique_districts, conflict_threshold=0.1)
        predicted_districts.append(pred_district)
        district_similarity_scores.append(district_sim)
        district_warnings.append(warning)
        
        # ----- Revenue Circle-Level Geocoding -----
        # If district is not conflicted, proceed to revenue circle matching
        if pred_district != "CONFLICT":
            # Filter the village list to rows matching the predicted district (using district_2)
            filtered_village = village_df[village_df["district_2"].apply(lambda x: clean_text(x)) == pred_district]
            if len(filtered_village) > 0:
                # Get unique revenue circles from the filtered list
                unique_revenue = filtered_village["revenue_ci"].dropna().unique().tolist()
                unique_revenue = [clean_text(x) for x in unique_revenue]
                if len(unique_revenue) > 0:
                    revenue_embeddings = compute_embeddings(model, unique_revenue)
                    pred_revenue, revenue_sim = geocode_revenue_circle(query_embedding, revenue_embeddings, unique_revenue)
                else:
                    pred_revenue, revenue_sim = "", 0
            else:
                pred_revenue, revenue_sim = "", 0
        else:
            pred_revenue, revenue_sim = "", 0
        
        predicted_revenue_circles.append(pred_revenue)
        revenue_similarity_scores.append(revenue_sim)
    
    # ----------------------------
    # Append Results to the DataFrame
    # ----------------------------
    schemes_df["predicted_district"] = predicted_districts
    schemes_df["district_similarity"] = district_similarity_scores
    schemes_df["district_warning"] = district_warnings
    schemes_df["predicted_revenue_circle"] = predicted_revenue_circles
    schemes_df["revenue_similarity"] = revenue_similarity_scores
    
    # ----------------------------
    # Save the Output
    # ----------------------------
    output_csv = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\schemes_geocoded_with_conflicts.csv"
    schemes_df.to_csv(output_csv, index=False)
    print(f"Geocoding complete. Output saved as {output_csv}")

if __name__ == "__main__":
    main()


Geocoding complete. Output saved as schemes_geocoded_with_conflicts.csv


In [ ]:

# ---------------------------------------------
# Main Geocoding Function
# ---------------------------------------------

def main():
    # ----------------------------
    # Load Data
    # ----------------------------
    # Path to your schemes CSV file (adjust as needed)
    schemes_csv = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_DDO_expenses.csv"  
    # Path to your master village list CSV file (assumed to have a column "village")
    villages_csv = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\ASSAM_VILLAGES_MASTER.csv"
    # Optionally, load the revenue circle boundary shapefile (if needed for further spatial join)
    revenue_circle_shp = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Maps\as_ids-drr_shapefiles\assam_rc_2024-11.geojson"  # adjust path as needed

    schemes_df = pd.read_csv(schemes_csv)
    villages_df = pd.read_csv(villages_csv)
    
    # If you need to incorporate revenue circle details, load the shapefile:
    revenue_circle_gdf = gpd.read_file(revenue_circle_shp)
    
    # ----------------------------
    # Prepare the Embedding Model
    # ----------------------------
    # Using a compact SentenceTransformer model
    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # Compute embeddings for the master village names
    # Assume your villages CSV has a column named "village"
    village_names = villages_df["village"].tolist()
    village_names_clean, village_embeddings = compute_village_embeddings(model, village_names)
    
    # ----------------------------
    # Define which fields to combine from schemes
    # ----------------------------
    text_fields = [
        "Name of the Scheme", 
        "Head office", 
        "Aim, objectives and benefit expected from the scheme/project", 
        "TREASURY NAME", 
        "DDO NAME"
    ]
    
    # Prepare lists to store geocoding results
    geocoded_villages = []
    similarity_scores = []
    
    # ----------------------------
    # Process Each Scheme Row
    # ----------------------------
    for idx, row in schemes_df.iterrows():
        # Combine key fields into one text string
        combined_text = combine_fields(row, text_fields)
        combined_text_clean = clean_text(combined_text)
        
        # Compute embedding for the combined text; shape will be (1, D)
        scheme_embedding = model.encode([combined_text_clean], convert_to_tensor=False)
        
        # Compare with village embeddings to find the best match
        best_village, best_similarity = geocode_scheme_text(scheme_embedding, village_embeddings, village_names_clean)
        
        geocoded_villages.append(best_village)
        similarity_scores.append(best_similarity)
    
    # Add the geocoding results to the schemes dataframe
    schemes_df["geocoded_village"] = geocoded_villages
    schemes_df["similarity_score"] = similarity_scores
    
    # ----------------------------
    # (Optional) Merge with Village Details
    # ----------------------------
    # If your master list has additional details (e.g., latitude, longitude, revenue circle),
    # merge them into the schemes dataframe.
    villages_df["village_clean"] = villages_df["village"].apply(clean_text)
    schemes_df = schemes_df.merge(villages_df, left_on="geocoded_village", right_on="village_clean", how="left")
    
    # ----------------------------
    # (Optional) Spatial Join with Revenue Circle Boundaries
    # ----------------------------
    # If your villages_df (or merged dataframe) has spatial coordinates and you wish to obtain
    # revenue circle details, you could perform a spatial join here.
    # For example, if villages_df had columns 'longitude' and 'latitude':
    #
    # villages_gdf = gpd.GeoDataFrame(villages_df, 
    #                 geometry=gpd.points_from_xy(villages_df.longitude, villages_df.latitude),
    #                 crs="EPSG:4326")
    # schemes_gdf = schemes_df.merge(villages_gdf[['village_clean','geometry']], left_on="geocoded_village", right_on="village_clean")
    # schemes_gdf = gpd.sjoin(schemes_gdf, revenue_circle_gdf, how="left", op="within")
    #
    # Adjust the spatial join as per your data schema.
    
    # ----------------------------
    # Save the Final Output
    # ----------------------------
    output_csv = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\Sources\SDRF\data\flood_schemes_geocoded.csv"
    schemes_df.to_csv(output_csv, index=False)
    print(f"Geocoding complete. Results saved to {output_csv}")

if __name__ == "__main__":
    main()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

KeyError: 'village'